ENVIRON

In [1]:
import networkx as nx
import pandas as pd
import os
from tqdm import tqdm, trange
from itertools import combinations

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
tqdm.pandas()

In [3]:
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

In [4]:
for index, model_option in enumerate(os.listdir('./datasets/')):
    # INDEXING
    print(f'[{index}/{len(os.listdir("./datasets/"))}] - {model_option}')
    if model_option == 'original':
        continue

    # DATASET - ROBLOX
    roblox1_df = pd.read_csv(f'./datasets/{model_option}/roblox1.csv', index_col = 0, low_memory = False)
    roblox2_df = pd.read_csv(f'./datasets/{model_option}/roblox2.csv', index_col = 0, low_memory = False)
    roblox3_df = pd.read_csv(f'./datasets/{model_option}/roblox3.csv', index_col = 0, low_memory = False)
    roblox4_df = pd.read_csv(f'./datasets/{model_option}/roblox4.csv', index_col = 0, low_memory = False)
    roblox5_df = pd.read_csv(f'./datasets/{model_option}/roblox5.csv', index_col = 0, low_memory = False)

    df = pd.concat([roblox1_df, roblox2_df, roblox3_df, roblox4_df, roblox5_df]).dropna(subset = ['keybert_keywords']).reset_index(drop = True)
    
    tqdm.pandas(desc = '\t(01/12)')
    df['keybert_keywords'] = df['keybert_keywords'].progress_apply(eval)
    
    tqdm.pandas(desc = '\t(02/12)')
    df['keywords'] = df['keybert_keywords'].progress_apply(lambda x : list(flatten(x))[0] if len(x) > 0 else '')

    # GRAPH FORMULATION - ROBLOX
    G = nx.MultiGraph()
    unique_items = df['keywords'].explode().unique()

    for item in tqdm(unique_items, desc = '\t(03/12)'):
        G.add_node(item, keyword = item)
    for keywords in tqdm(df['keywords'], desc = '\t(04/12)'):
        pairs = list(combinations(keywords, 2))
        pairs = [(pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0]) for pair in pairs]
        
        G.add_edges_from(pairs)

    # MULTI to SINGLE - ROBLOX
    H = nx.Graph()
    unique_items = df['keywords'].explode().unique()

    for item in tqdm(unique_items, desc = '\t(05/12)'):
        H.add_node(item, keyword = item)
    for u, v, data in tqdm(G.edges(data = True), desc = '\t(06/12)'):
        w = data['weight'] if 'weight' in data else 1.0
        if H.has_edge(u, v):
            H[u][v]['weight'] += w
        else:
            H.add_edge(u, v, weight=w)
    nx.write_graphml_lxml(H, f'./graph/roblox-{model_option}.graphml')

    # DATASET - ZEPETO
    df = pd.read_csv(f'./datasets/{model_option}/zepeto.csv', index_col = 0, low_memory = False)
    
    tqdm.pandas(desc = '\t(07/12)')
    df['keybert_keywords'] = df['keybert_keywords'].progress_apply(eval)
    
    tqdm.pandas(desc = '\t(08/12)')
    df['keywords'] = df['keybert_keywords'].progress_apply(lambda x : list(flatten(x))[0] if len(x) > 0 else '')

    # GRAPH FORMULATION - ZEPETO
    G = nx.MultiGraph()
    unique_items = df['keywords'].explode().unique()

    for item in tqdm(unique_items, desc = '\t(09/12)'):
        G.add_node(item, keyword = item)
    for keywords in tqdm(df['keywords'], desc = '\t(10/12)'):
        pairs = list(combinations(keywords, 2))
        pairs = [(pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0]) for pair in pairs]
        
        G.add_edges_from(pairs)

    # MULTI to SINGLE - ZEPETO
    H = nx.Graph()
    unique_items = df['keywords'].explode().unique()

    for item in tqdm(unique_items, desc = '\t(11/12)'):
        H.add_node(item, keyword = item)
    for u, v, data in tqdm(G.edges(data = True), desc = '\t(12/12)'):
        w = data['weight'] if 'weight' in data else 1.0
        if H.has_edge(u, v):
            H[u][v]['weight'] += w
        else:
            H.add_edge(u, v, weight=w)
    nx.write_graphml_lxml(H, f'./graph/zepeto-{model_option}.graphml')

[0/18] - keybert-all-MiniLM-L12-v2-post-tokenized


	(12/12): 100%|██████████| 1717869/1717869 [00:03<00:00, 442570.35it/s]


[1/18] - keybert-all-MiniLM-L12-v2-tokenized


	(12/12): 100%|██████████| 1594633/1594633 [00:03<00:00, 461190.75it/s]


[2/18] - keybert-all-MiniLM-L12-v2


...

[15/18] - keybert-multi-qa-mpnet-base-dot-v1-tokenized


	(12/12): 100%|██████████| 1594633/1594633 [00:03<00:00, 442315.12it/s]


[16/18] - keybert-multi-qa-mpnet-base-dot-v1


	(12/12): 100%|██████████| 1792217/1792217 [00:03<00:00, 465985.04it/s]


[17/18] - original
